In [ ]:
# pip install domolibrary

In [ ]:
import os

domo_username = os.environ.get("DOMO_USERNAME")
domo_instance = "domo-community"
domo_password = os.environ.get("DOJO_PASSWORD")

In [ ]:
import domolibrary.client.DomoAuth as dmda

auth = dmda.DomoFullAuth(
    domo_username=domo_username,
    domo_password=domo_password,
    domo_instance=domo_instance,
)

await auth.print_is_token()

🎉 full_auth token retrieved from domo-community ⚙️


True

In [ ]:
import domolibrary.classes.DomoDatacenter as dmdc

domo_datacenter = dmdc.DomoDatacenter(auth=auth)

dataflows_ls = await domo_datacenter.search_datacenter(
    auth=auth,
    entity_type="DATAFLOW",
    additional_filters_ls=[
        {
            "filterType": "term",
            "field": "data_flow_type",
            "value": "MAGIC",
            "name": "Magic ETL v2",
            "not": False,
        }
    ],
)

dataflows_ls[0]

{'entityType': 'dataflow',
 'databaseId': '340',
 'searchId': {'indexName': None,
  'databaseId': '340',
  'customer': 'mmmm-0012-0200',
  'entityType': 'dataflow'},
 'createDate': 1699938646000,
 'lastModified': 1701451542000,
 'lastIndexed': 1701451545056,
 'highlightedFields': {},
 'language': 'English',
 'requestAccess': False,
 'score': 56.957905,
 'name': 'Demo Active Employees',
 'description': '',
 'ownedById': '699075231',
 'ownedByName': 'Mark Snodgrass',
 'tags': [],
 'lastRunDate': 1699938659000,
 'inputDatasets': [{'name': 'DomoStats - People',
   'id': '241025d7-3cca-4369-b7c0-b3264277c0e1'},
  {'name': 'Domo Dimensions - Calendar',
   'id': 'd6b99a99-63c7-46f0-8e54-38045e29e710'}],
 'outputDatasets': [{'name': 'Demo Active Employees',
   'id': 'b25b7f48-5257-4e26-bffa-b240b0caefea'}],
 'runCount': 1,
 'successRate': 1.0,
 'dataFlowType': 'MAGIC',
 'status': 'SUCCESS',
 'deleted': False,
 'passwordProtected': False,
 'abandoned': False,
 'owners': [{'id': '699075231',
   

In [ ]:
dataflow_ids = [dataflow.get("databaseId") for dataflow in dataflows_ls]
dataflow_ids[0:5]

['340', '131', '227', '37', '185']

In [ ]:
import domolibrary.classes.DomoDataflow as dmdf
import asyncio

dataflow_definition_ls = await asyncio.gather(
    *[
        dmdf.DomoDataflow.get_from_id(dataflow_id=dataflow_id, auth=auth)
        for dataflow_id in dataflow_ids
    ]
)

domo_dataflow = dataflow_definition_ls[0]
domo_dataflow

DomoDataflow(id=340, name='Demo Active Employees', owner=None, description=None, tags=None, actions=[DomoDataflow_Action(id='d7b935c6-2184-4079-bc7b-4e50e25d1801', type='LoadFromVault', name='DomoStats - People', datasource_id='241025d7-3cca-4369-b7c0-b3264277c0e1', sql=None, depends_on=None, parent_actions=None), DomoDataflow_Action(id='cd3d5f9e-0bfe-40d9-bb95-45faf2d64b30', type='SelectValues', name='Select Columns', datasource_id=None, sql=None, depends_on=['d7b935c6-2184-4079-bc7b-4e50e25d1801'], parent_actions=[DomoDataflow_Action(id='d7b935c6-2184-4079-bc7b-4e50e25d1801', type='LoadFromVault', name='DomoStats - People', datasource_id='241025d7-3cca-4369-b7c0-b3264277c0e1', sql=None, depends_on=None, parent_actions=None)]), DomoDataflow_Action(id='18e365d8-8b24-471b-a1da-694a1936ad5c', type='LoadFromVault', name='Domo Dimensions - Calendar', datasource_id='d6b99a99-63c7-46f0-8e54-38045e29e710', sql=None, depends_on=None, parent_actions=None), DomoDataflow_Action(id='db088785-df3f-

In [ ]:
domo_dataflow = await dmdf.DomoDataflow.get_from_id(dataflow_id=9, auth=auth)
domo_dataflow

DomoDataflow(id=9, name='DOMO Covid Datasets', owner=None, description=None, tags=None, actions=[DomoDataflow_Action(id='0e192d7e-280b-4c4d-bde2-f0f3adb1e213', type='LoadFromVault', name='DOMO Covid Current Snapshot Tracker Data', datasource_id='b034f19e-16d5-4cb3-afac-b72c8e6a79fd', sql=None, depends_on=None, parent_actions=None), DomoDataflow_Action(id='96d04e4b-dc72-479c-b84b-ad0b74ada085', type='LoadFromVault', name='DOMO Covid Time Series Tracker Data', datasource_id='dcad2f50-e65e-4259-a9e8-214a3d1e18a7', sql=None, depends_on=None, parent_actions=None), DomoDataflow_Action(id='a1d07083-d1ca-41e9-8be2-f13596df03c3', type='ValueMapper', name='Value Mapper', datasource_id=None, sql=None, depends_on=['96d04e4b-dc72-479c-b84b-ad0b74ada085'], parent_actions=[DomoDataflow_Action(id='96d04e4b-dc72-479c-b84b-ad0b74ada085', type='LoadFromVault', name='DOMO Covid Time Series Tracker Data', datasource_id='dcad2f50-e65e-4259-a9e8-214a3d1e18a7', sql=None, depends_on=None, parent_actions=None)]

In [ ]:
import pandas as pd


async def generate_version_action_pdf(domo_dataflow):
    dataflow_versions = await domo_dataflow.get_versions()

    df = pd.DataFrame(
        [
            {
                "dataflow_id": dataflow_version.id,
                "dataflow_version": dataflow_version.version_id,
                **domo_action.__dict__,
            }
            for dataflow_version in dataflow_versions
            for domo_action in dataflow_version.actions
        ]
    )

    df.drop(columns=["parent_actions", "datasource_id", "sql"], inplace=True)
    df.rename(columns={"id": "tile_id", "type": "tile_type"}, inplace=True)

    return df


action_df = await generate_version_action_pdf(domo_dataflow)
action_df[0:5]

,dataflow_id,dataflow_version,tile_id,tile_type,name,depends_on
0,9,89,0e192d7e-280b-4c4d-bde2-f0f3adb1e213,LoadFromVault,DOMO Covid Current Snapshot Tracker Data,None
1,9,89,96d04e4b-dc72-479c-b84b-ad0b74ada085,LoadFromVault,DOMO Covid Time Series Tracker Data,None
2,9,89,a1d07083-d1ca-41e9-8be2-f13596df03c3,ValueMapper,Value Mapper,[96d04e4b-dc72-479c-b84b-ad0b74ada085]
3,9,89,6e3ec9a4-470b-4fe4-8e8a-bd954b681df4,DateCalculator,Days Since...,[0e192d7e-280b-4c4d-bde2-f0f3adb1e213]
4,9,89,59e6226d-507f-4cff-86b1-f024499173aa,NumericCalculator,Resolved Cases,[6e3ec9a4-470b-4fe4-8e8a-bd954b681df4]


In [ ]:
async def generate_action_stats_df(domo_dataflow):
    await domo_dataflow.history.get_execution_history(maximum=10)

    df = pd.DataFrame(
        [
            {
                "dataflow_version": domo_history.dataflow_version,
                "dataflow_id": domo_history.dataflow_id,
                "dataflow_begin_time": domo_history.begin_time,
                **domo_action.__dict__,
            }
            for domo_history in domo_dataflow.history.execution_history
            for domo_action in domo_history.action_results
        ]
    )

    df.rename(columns={"id": "tile_id", "type": "tile_type"}, inplace=True)

    return df


stats_df = await generate_action_stats_df(domo_dataflow)
stats_df[0:5]

,dataflow_version,dataflow_id,dataflow_begin_time,tile_id,tile_type,name,is_success,rows_processed,begin_time,end_time,duration_in_sec
0,89,9,2024-02-15 21:46:47,0e192d7e-280b-4c4d-bde2-f0f3adb1e213,DataHubManifestLoaderAction,None,True,315,2024-02-15 21:46:57.697,2024-02-15 21:46:58.874,1.177
1,89,9,2024-02-15 21:46:47,96d04e4b-dc72-479c-b84b-ad0b74ada085,DataHubManifestLoaderAction,None,True,389137,2024-02-15 21:46:57.696,2024-02-15 21:47:06.790,9.094
2,89,9,2024-02-15 21:46:47,a1d07083-d1ca-41e9-8be2-f13596df03c3,ValueMapper,None,True,389137,2024-02-15 21:46:58.773,2024-02-15 21:47:06.790,8.017
3,89,9,2024-02-15 21:46:47,6e3ec9a4-470b-4fe4-8e8a-bd954b681df4,DateCalculator,None,True,315,2024-02-15 21:46:58.772,2024-02-15 21:46:58.873,0.101
4,89,9,2024-02-15 21:46:47,59e6226d-507f-4cff-86b1-f024499173aa,NumericCalculator,None,True,315,2024-02-15 21:46:58.773,2024-02-15 21:46:58.873,0.100


In [ ]:
def combine_execution_plan_with_stats_df(action_df, stats_df):
    stats_df.drop(columns=["tile_type", "name"], inplace=True)

    df = pd.merge(
        action_df,
        stats_df,
        on=["dataflow_version", "dataflow_id", "tile_id"],
        how="outer",
    )

    return df


combine_df = combine_execution_plan_with_stats_df(action_df.copy(), stats_df.copy())

print(len(combine_df))
combine_df[:10]

931


,dataflow_id,dataflow_version,tile_id,tile_type,name,depends_on,dataflow_begin_time,is_success,rows_processed,begin_time,end_time,duration_in_sec
0,9,20,008ac1ce-4143-420f-93a7-9536ea986fc5,WindowAction,Rank & Window,[e667577a-8c73-4677-8022-8bf3f3e7a72e],NaT,NaN,NaN,NaT,NaT,NaN
1,9,20,02a1f8dd-2b0a-4193-9c54-8d4fc19261cc,DateCalculator,Snapshot Age 1,[250bf67d-8882-4078-a7d4-aa6dda1a1635],NaT,NaN,NaN,NaT,NaT,NaN
2,9,20,0362dd20-2039-403d-a249-94e2ce716aed,Constant,Today,[18626a7b-c1a7-4fb4-bcc0-7e50ebd7e763],NaT,NaN,NaN,NaT,NaT,NaN
3,9,20,06f8d60b-8b84-474d-91de-abcaefe10abf,SelectValues,DIM_Country (Join),[0e783798-e39f-49f2-ac42-70cd82ffd611],NaT,NaN,NaN,NaT,NaT,NaN
4,9,20,08327437-2a70-49bb-9676-8e63f540eb84,ValueMapper,Null Recovered 2,[5da719b4-d4c6-4dfc-a928-38312de82df8],NaT,NaN,NaN,NaT,NaT,NaN
5,9,20,0c791afe-0acb-4821-8781-ddd5d51c850a,WindowAction,Recovered,[2037eb8f-c88c-4d9f-8d14-ae591a27b627],NaT,NaN,NaN,NaT,NaT,NaN
6,9,20,0e192d7e-280b-4c4d-bde2-f0f3adb1e213,LoadFromVault,DOMO Covid Current Snapshot Tracker Data,None,NaT,NaN,NaN,NaT,NaT,NaN
7,9,20,0e783798-e39f-49f2-ac42-70cd82ffd611,UnionAll,Append New Values,"[3ea82c4a-1da1-4c37-b3e5-ea77a6ba035d, 52b67a4...",NaT,NaN,NaN,NaT,NaT,NaN
8,9,20,1689d811-85eb-4e3a-993d-2e59419e2236,SelectValues,DIM Table,[d6423ad7-938f-428e-a773-c8f2f257de07],NaT,NaN,NaN,NaT,NaT,NaN
9,9,20,16bae147-462b-4547-94b2-8075aab0a2fb,WindowAction,Deaths 1,[551ebf46-d0ac-4210-83cd-a50049b2b61d],NaT,NaN,NaN,NaT,NaT,NaN


In [ ]:
def explode_parents(df):
    """
    1. generates one row for each parent in "depends_on"
    2. joins parents stats to df
    3. stores dataflow_begin_time if there is no parent
    """

    df["dependencies"] = df["depends_on"]
    explode_df = df.explode("depends_on")

    parents_df = df[
        [
            "dataflow_version",
            "dataflow_id",
            "tile_id",
            "tile_type",
            "name",
            "end_time",
            "rows_processed",
        ]
    ].rename(
        columns={
            "tile_id": "parent_tile_id",
            "tile_type": "parent_tile_type",
            "name": "parent_tile_name",
            "end_time": "parent_end_time",
            "rows_processed": "parent_rows_processed",
        }
    )

    explode_df = pd.merge(
        explode_df,
        parents_df,
        left_on=["dataflow_version", "dataflow_id", "depends_on"],
        right_on=["dataflow_version", "dataflow_id", "parent_tile_id"],
        how="left",
    )

    explode_df["parent_end_time"] = explode_df[["begin_time", "parent_end_time"]].max(
        axis=1
    )

    explode_df["actual_duration_in_sec"] = (
        explode_df["end_time"] - explode_df["parent_end_time"]
    ).dt.total_seconds()

    explode_df = explode_df[
        [
            "dataflow_id",
            "dataflow_version",
            "tile_id",
            "tile_type",
            "name",
            "rows_processed",
            "begin_time",
            "end_time",
            "duration_in_sec",
            "parent_end_time",
            "actual_duration_in_sec",
            "dependencies",
            "parent_tile_id",
            "parent_tile_type",
            "parent_tile_name",
            "parent_rows_processed",
        ]
    ]

    explode_df["parent_tile_id"].fillna("start", inplace=True)

    return explode_df


explode_df = explode_parents(combine_df.copy())
explode_df[0:10]

/tmp/ipykernel_36055/2268720893.py:68: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  explode_df["parent_tile_id"].fillna("start", inplace=True)


,dataflow_id,dataflow_version,tile_id,tile_type,name,rows_processed,begin_time,end_time,duration_in_sec,parent_end_time,actual_duration_in_sec,dependencies,parent_tile_id,parent_tile_type,parent_tile_name,parent_rows_processed
0,9,20,008ac1ce-4143-420f-93a7-9536ea986fc5,WindowAction,Rank & Window,NaN,NaT,NaT,NaN,NaT,NaN,[e667577a-8c73-4677-8022-8bf3f3e7a72e],e667577a-8c73-4677-8022-8bf3f3e7a72e,UnionAll,Append Unused DIM Countries,NaN
1,9,20,02a1f8dd-2b0a-4193-9c54-8d4fc19261cc,DateCalculator,Snapshot Age 1,NaN,NaT,NaT,NaN,NaT,NaN,[250bf67d-8882-4078-a7d4-aa6dda1a1635],250bf67d-8882-4078-a7d4-aa6dda1a1635,NumericCalculator,Active Cases 2,NaN
2,9,20,0362dd20-2039-403d-a249-94e2ce716aed,Constant,Today,NaN,NaT,NaT,NaN,NaT,NaN,[18626a7b-c1a7-4fb4-bcc0-7e50ebd7e763],18626a7b-c1a7-4fb4-bcc0-7e50ebd7e763,Filter,Countries Still Listed,NaN
3,9,20,06f8d60b-8b84-474d-91de-abcaefe10abf,SelectValues,DIM_Country (Join),NaN,NaT,NaT,NaN,NaT,NaN,[0e783798-e39f-49f2-ac42-70cd82ffd611],0e783798-e39f-49f2-ac42-70cd82ffd611,UnionAll,Append New Values,NaN
4,9,20,08327437-2a70-49bb-9676-8e63f540eb84,ValueMapper,Null Recovered 2,NaN,NaT,NaT,NaN,NaT,NaN,[5da719b4-d4c6-4dfc-a928-38312de82df8],5da719b4-d4c6-4dfc-a928-38312de82df8,ValueMapper,Null Deaths 2,NaN
5,9,20,0c791afe-0acb-4821-8781-ddd5d51c850a,WindowAction,Recovered,NaN,NaT,NaT,NaN,NaT,NaN,[2037eb8f-c88c-4d9f-8d14-ae591a27b627],2037eb8f-c88c-4d9f-8d14-ae591a27b627,WindowAction,Deaths,NaN
6,9,20,0e192d7e-280b-4c4d-bde2-f0f3adb1e213,LoadFromVault,DOMO Covid Current Snapshot Tracker Data,NaN,NaT,NaT,NaN,NaT,NaN,None,start,NaN,NaN,NaN
7,9,20,0e783798-e39f-49f2-ac42-70cd82ffd611,UnionAll,Append New Values,NaN,NaT,NaT,NaN,NaT,NaN,"[3ea82c4a-1da1-4c37-b3e5-ea77a6ba035d, 52b67a4...",3ea82c4a-1da1-4c37-b3e5-ea77a6ba035d,Constant,Flag New Values,NaN
8,9,20,0e783798-e39f-49f2-ac42-70cd82ffd611,UnionAll,Append New Values,NaN,NaT,NaT,NaN,NaT,NaN,"[3ea82c4a-1da1-4c37-b3e5-ea77a6ba035d, 52b67a4...",52b67a4e-c879-432d-95f4-ccf64475f1b7,SelectValues,Up to Date Covid + Existing Webform Info,NaN
9,9,20,0e783798-e39f-49f2-ac42-70cd82ffd611,UnionAll,Append New Values,NaN,NaT,NaT,NaN,NaT,NaN,"[3ea82c4a-1da1-4c37-b3e5-ea77a6ba035d, 52b67a4...",97044bf5-caa3-4420-b40f-9002d836d5c5,SelectValues,Existing Webform Info - No Longer in Covid,NaN


In [ ]:
def calculate_average(df):
    """calculates average execution time for each tile, differentiating by reported duration_in_sec vs actual_duration which assumes an upstream parent"""

    return df.groupby(
        ["dataflow_id", "dataflow_version", "tile_id", "parent_tile_id"]
    ).agg(
        {
            "name": "first",
            "tile_type": "first",
            "parent_tile_name": "first",
            "parent_tile_type": "first",
            "duration_in_sec": "mean",
            "actual_duration_in_sec": "mean",
            "rows_processed": "mean",
            "parent_rows_processed": "mean",
        }
    )


calculate_average(explode_df.copy())

name  \
dataflow_id dataflow_version tile_id                              parent_tile_id                                             
9           20               008ac1ce-4143-420f-93a7-9536ea986fc5 e667577a-8c73-4677-8022-8bf3f3e7a72e       Rank & Window   
                             02a1f8dd-2b0a-4193-9c54-8d4fc19261cc 250bf67d-8882-4078-a7d4-aa6dda1a1635      Snapshot Age 1   
                             0362dd20-2039-403d-a249-94e2ce716aed 18626a7b-c1a7-4fb4-bcc0-7e50ebd7e763               Today   
                             06f8d60b-8b84-474d-91de-abcaefe10abf 0e783798-e39f-49f2-ac42-70cd82ffd611  DIM_Country (Join)   
                             08327437-2a70-49bb-9676-8e63f540eb84 5da719b4-d4c6-4dfc-a928-38312de82df8    Null Recovered 2   
...                                                                                                                    ...   
            89               d652deb2-cd3e-462d-9627-2f72425e05cf 08327437-2a70-49bb-9676-8e63f540eb84  Null New Recovered   
                             dac9af4e-a203-47f5-9493-ec46d834a51f cebf2fb3-75e3-48ee-bfe5-2af9927cb581    Resolved Cases 2   
                             dc5a2d0c-3bdf-4e94-a07a-7ff4709acc1f 1d3e790a-dabf-45ba-849e-74996d7a03c9      Active Cases 1   
                             eeecc67b-34c4-4929-a4e3-34058aba039a 43029ef0-9519-49c6-80c2-2f885e662221             Ranks 1   
                             fe46f53d-d10e-4e1e-b341-4d18f565e83b d652deb2-cd3e-462d-9627-2f72425e05cf       Null or 0 Lat   

                                                                                                                tile_type  \
dataflow_id dataflow_version tile_id                              parent_tile_id                                            
9           20               008ac1ce-4143-420f-93a7-9536ea986fc5 e667577a-8c73-4677-8022-8bf3f3e7a72e       WindowAction   
                             02a1f8dd-2b0a-4193-9c54-8d4fc19261cc 250bf67d-8882-4078-a7d4-aa6dda1a1635     DateCalculator   
                             0362dd20-2039-403d-a249-94e2ce716aed 18626a7b-c1a7-4fb4-bcc0-7e50ebd7e763           Constant   
                             06f8d60b-8b84-474d-91de-abcaefe10abf 0e783798-e39f-49f2-ac42-70cd82ffd611       SelectValues   
                             08327437-2a70-49bb-9676-8e63f540eb84 5da719b4-d4c6-4dfc-a928-38312de82df8        ValueMapper   
...                                                                                                                   ...   
            89               d652deb2-cd3e-462d-9627-2f72425e05cf 08327437-2a70-49bb-9676-8e63f540eb84        ValueMapper   
                             dac9af4e-a203-47f5-9493-ec46d834a51f cebf2fb3-75e3-48ee-bfe5-2af9927cb581  NumericCalculator   
                             dc5a2d0c-3bdf-4e94-a07a-7ff4709acc1f 1d3e790a-dabf-45ba-849e-74996d7a03c9  NumericCalculator   
                             eeecc67b-34c4-4929-a4e3-34058aba039a 43029ef0-9519-49c6-80c2-2f885e662221       SelectValues   
                             fe46f53d-d10e-4e1e-b341-4d18f565e83b d652deb2-cd3e-462d-9627-2f72425e05cf        ValueMapper   

                                                                                                                   parent_tile_name  \
dataflow_id dataflow_version tile_id                              parent_tile_id                                                      
9           20               008ac1ce-4143-420f-93a7-9536ea986fc5 e667577a-8c73-4677-8022-8bf3f3e7a72e  Append Unused DIM Countries   
                             02a1f8dd-2b0a-4193-9c54-8d4fc19261cc 250bf67d-8882-4078-a7d4-aa6dda1a1635               Active Cases 2   
                             0362dd20-2039-403d-a249-94e2ce716aed 18626a7b-c1a7-4fb4-bcc0-7e50ebd7e763       Countries Still Listed   
                             06f8d60b-8b84-474d-91de-abcaefe10abf 0e783798-e39f-49f2-ac42-70cd82ffd611            Append New Values   
                             08327437-2a70-

In [ ]:
def get_action_list(domo_dataflow: dmdf.DomoDataflow):
    return [
        {
            "dataflow_id": domo_dataflow.id,
            "datafow_name": domo_dataflow.name,
            "action_type": domo_action.type,
        }
        for domo_action in domo_dataflow.actions
    ]


get_action_list(domo_dataflow=dataflow_definition_ls[0])

[{'dataflow_id': 340,
  'datafow_name': 'Demo Active Employees',
  'action_type': 'LoadFromVault'},
 {'dataflow_id': 340,
  'datafow_name': 'Demo Active Employees',
  'action_type': 'SelectValues'},
 {'dataflow_id': 340,
  'datafow_name': 'Demo Active Employees',
  'action_type': 'LoadFromVault'},
 {'dataflow_id': 340,
  'datafow_name': 'Demo Active Employees',
  'action_type': 'Filter'},
 {'dataflow_id': 340,
  'datafow_name': 'Demo Active Employees',
  'action_type': 'SelectValues'},
 {'dataflow_id': 340,
  'datafow_name': 'Demo Active Employees',
  'action_type': 'Constant'},
 {'dataflow_id': 340,
  'datafow_name': 'Demo Active Employees',
  'action_type': 'Constant'},
 {'dataflow_id': 340,
  'datafow_name': 'Demo Active Employees',
  'action_type': 'MergeJoin'},
 {'dataflow_id': 340,
  'datafow_name': 'Demo Active Employees',
  'action_type': 'Filter'},
 {'dataflow_id': 340,
  'datafow_name': 'Demo Active Employees',
  'action_type': 'SelectValues'},
 {'dataflow_id': 340,
  'datafo

In [ ]:
action_ls = [get_action_list(dd_obj) for dd_obj in dataflow_definition_ls]

flat_action_ls = [action for a_list in action_ls for action in a_list]
flat_action_ls[0:5]

[{'dataflow_id': 340,
  'datafow_name': 'Demo Active Employees',
  'action_type': 'LoadFromVault'},
 {'dataflow_id': 340,
  'datafow_name': 'Demo Active Employees',
  'action_type': 'SelectValues'},
 {'dataflow_id': 340,
  'datafow_name': 'Demo Active Employees',
  'action_type': 'LoadFromVault'},
 {'dataflow_id': 340,
  'datafow_name': 'Demo Active Employees',
  'action_type': 'Filter'},
 {'dataflow_id': 340,
  'datafow_name': 'Demo Active Employees',
  'action_type': 'SelectValues'}]

In [ ]:
import pandas as pd

df = pd.DataFrame(flat_action_ls)
df[0:5]

,dataflow_id,datafow_name,action_type
0,340,Demo Active Employees,LoadFromVault
1,340,Demo Active Employees,SelectValues
2,340,Demo Active Employees,LoadFromVault
3,340,Demo Active Employees,Filter
4,340,Demo Active Employees,SelectValues


In [ ]:
df.to_csv("marks_csv.csv")